In [1]:
# Benders decomposition for Fixed Charge Transportation Problem

import gurobipy as gp
from gurobipy import *
import numpy as np

In [2]:
#part a)
# solve with example in lecture
capacity = [5,8,7]
demand = [3,6,6,5]

locations = len(capacity)
customers = len(demand)

varcost = [[4,5,2,7],[5,8,6,2],[8,9,4,3]]
fixcost = [[2,10,1,5],[7,2,7,8],[4,0,3,9]]

bigM = [[min(capacity[i], demand[j]) for j in range(customers)] for i in range(locations)]

In [3]:
# Define Benders subproblem
        
def subproblem(model, where):
    if where == GRB.Callback.MIPSOL:
        v_y = model.cbGetSolution(model._y)
        #print('\nCurrent y: ', v_y)
        
        LB = model.cbGetSolution(model._z)
        print('Current LB: ', LB, '\n')
        print(model.cbGet(GRB.Callback.MIPSOL_OBJBND))
        print(model.cbGet(GRB.Callback.MIPSOL_OBJBST))
        
        bsp = gp.Model("Subproblem") 
        #variables
        x = {}
        for i in range(locations):
            for j in range(customers):
                x[i,j] = bsp.addVar(obj=varcost[i][j])
        
        #unserved demand
        s = {}
        add_cost = np.max(fixcost)*2
        for j in range(customers):
            s[j] = bsp.addVar(obj = add_cost)
        
        #demand constraint
        demand_constr = {} 
        for j in range(customers):
            demand_constr[j] = bsp.addConstr(sum(x[i,j] for i in range(locations)) + s[j] == demand[j])
        
        #capacity constraint
        cap_constr = {}
        for i in range(locations):
            cap_constr[i] = bsp.addConstr(sum(x[i,j] for j in range(customers)) <= capacity[i])
        
        #logical constraint
        log_constr = {}
        for i in range(locations):
            for j in range(customers):
                log_constr[i,j] = bsp.addConstr(x[i,j] <= v_y[i,j] * bigM[i][j])
        
        bsp.optimize()
        
        
        #update ub and lb
        fc = 0
        for i in range(locations):
            for j in range(customers):
                if x[i,j].x > 0: 
                    fc+=fixcost[i][j]
                
        v = np.zeros(customers) #dual of demand constraints
        for j in range(customers):
            v[j] = demand_constr[j].pi #get dual value

        w = np.zeros((locations,customers)) #dual of logical constraints
        for i in range(locations):
            for j in range(customers):
                w[i,j] = log_constr[i,j].pi

        u = np.zeros(locations) #dual of capacity constraints
        for i in range(locations):
            u[i] = cap_constr[i].pi

        cm = np.zeros((locations,customers)) #coefficient of y in master problem
        for i in range(locations):
            for j in range(customers):
                cm[i,j] = fixcost[i][j] + bigM[i][j] * w[i,j]


        model.cbLazy(model._z >= sum(demand[j] * v[j] for j in range(customers)) +   
                     sum(capacity[i] * u[i] for i in range(locations)) + 
                     sum(cm[i,j] * model._y[i,j] for i in range(locations) for j in range(customers)))

        
        
        # save result of x from subproblem
        global transport
        for i in range(locations):
            for j in range(customers):
                transport[i,j] = x[i,j].x
        #print(transport)
        print([s[j].x for j in range(customers)])
        #print(bsp.objVal)

        bsp.dispose()
                      

In [4]:
def masterproblem(MIPGap = None, TimeLimit = None):
    global m
    
    # Relaxed Master problem function
    y = {}
    z = m.addVar(obj=1)                                                                           
    for i in range(locations):
        for j in range(customers):
            y[i,j] = m.addVar(vtype=GRB.BINARY)

    m.Params.lazyConstraints = 1
    m._y = y
    m._z = z
    
    if MIPGap is not None: m.Params.MIPGap = MIPGap
    if TimeLimit is not None: m.Params.TimeLimit = TimeLimit
    m.optimize(subproblem)


    print("Objective: " + str(m.objVal))

    print("Solution:")
    import pandas as pd
    index = ['Source ' + str(x) for x in range(locations)]
    columns = ['Customer ' + str(x) for x in range(customers)]

    solution = pd.DataFrame(index=index, columns=columns)

    for i in range(locations):
        for j in range(customers):
            solution.iloc[i,j] = transport[i,j]

    print(solution)
    print([y[i,j].x for i in range(locations) for j in range(customers)])
    #m.dispose()

In [5]:
# solve problem with Benders decomposition
transport = np.zeros((locations, customers)) # to save result of x from subproblem
m = gp.Model("Benders Fixed Charge Transportation Masterproblem") # to save model
masterproblem()

Using license file C:\Users\hadao\gurobi.lic
Academic license - for non-commercial use only
Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 0 rows, 13 columns and 0 nonzeros
Model fingerprint: 0xd03868d4
Variable types: 1 continuous, 12 integer (12 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Current LB:  0.0 

-1e+100
0.0
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 19 rows, 16 columns and 40 nonzeros
Model fingerprint: 0xf57eade9
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+00, 8e+00]
Presolve removed 19 rows and 16 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Prim

Optimize a model with 19 rows, 16 columns and 40 nonzeros
Model fingerprint: 0xda28e1c7
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+00, 8e+00]
Presolve removed 16 rows and 11 columns
Presolve time: 0.01s
Presolved: 3 rows, 5 columns, 7 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.8000000e+01   2.500000e+00   0.000000e+00      0s
       1    9.3000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.01 seconds
Optimal objective  9.300000000e+01
[0.0, 0.0, 0.0, 0.0]
Current LB:  117.0 

28.222222222222225
117.0
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 19 rows, 16 columns and 40 nonzeros
Model fingerprint: 0x109ea305
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+00, 8e+00]
Presolve re

Best objective 1.090000000000e+02, best bound 1.090000000000e+02, gap 0.0000%

User-callback calls 96, time in user-callback 0.30 sec
Objective: 109.00000000000001
Solution:
          Customer 0  Customer 1  Customer 2  Customer 3
Source 0           3           0           2           0
Source 1           0           3           0           5
Source 2           0           3           4           0
[1.0, -0.0, 1.0, -0.0, -0.0, 1.0, -0.0, 1.0, 0.0, 1.0, 1.0, 0.0]


In [6]:
#part b)
# solve with large example in database

In [7]:
file = open('36.txt','r')
data = file.readlines()

[locations, customers] = [int(x) for x in data[0].split(' ')]
capacity = [int(data[1].split(' ')[i]) for i in range(locations) ]
demand = [int(data[2].split(' ')[j]) for j in range(customers) ]
fixcost = [ [ int(data[i+3].split('   ')[j]) for j in range(customers) ] for i in range(locations)]
varcost = [ [ int(x) for x in data[i+3+locations+1].split('    ') if x!=''] for i in range(locations)]

print(locations)
print(customers)
print(capacity)
print(demand)
varcost

15
15
[20, 18, 17, 15, 15, 10, 10, 10, 9, 8, 6, 5, 5, 4, 4]
[20, 17, 16, 15, 13, 12, 11, 10, 10, 9, 7, 7, 7, 1, 1]


[[27, 35, 19, 32, 16, 19, 21, 21, 10, 12, 32, 14, 31, 10, 14],
 [20, 23, 20, 25, 24, 13, 21, 17, 23, 37, 16, 7, 12, 26, 35],
 [34, 22, 20, 12, 24, 29, 31, 9, 36, 27, 15, 38, 14, 33, 27],
 [17, 38, 23, 9, 34, 20, 29, 16, 26, 27, 24, 12, 27, 18, 27],
 [14, 35, 14, 27, 14, 24, 33, 31, 12, 34, 19, 31, 36, 33, 31],
 [27, 27, 23, 30, 30, 26, 22, 36, 34, 29, 31, 23, 27, 32, 25],
 [21, 30, 19, 14, 13, 24, 8, 15, 21, 31, 19, 26, 9, 13, 11],
 [9, 32, 24, 33, 23, 33, 24, 15, 30, 26, 9, 28, 37, 20, 18],
 [14, 31, 34, 23, 22, 9, 20, 37, 13, 9, 12, 38, 7, 27, 28],
 [30, 20, 23, 11, 35, 32, 32, 13, 34, 29, 27, 18, 30, 35, 30],
 [19, 32, 11, 17, 28, 28, 18, 9, 37, 9, 28, 14, 9, 13, 36],
 [29, 25, 30, 32, 24, 10, 28, 29, 18, 12, 13, 10, 29, 26, 8],
 [28, 12, 34, 25, 34, 33, 21, 19, 20, 21, 25, 26, 34, 21, 35],
 [28, 28, 11, 21, 9, 33, 9, 21, 13, 10, 23, 27, 22, 10, 36],
 [18, 19, 27, 32, 36, 30, 14, 21, 16, 15, 21, 34, 17, 31, 30]]

In [8]:
bigM = [[min(capacity[i], demand[j]) for j in range(customers)] for i in range(locations)]

In [ ]:
# solve problem with Benders decomposition
transport = np.zeros((locations, customers)) # to save result of x from subproblem
m = gp.Model("Benders Fixed Charge Transportation Masterproblem") # to save model
masterproblem(TimeLimit = 3600)

Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter TimeLimit to 5400.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 0 rows, 226 columns and 0 nonzeros
Model fingerprint: 0x01ab2a9e
Variable types: 1 continuous, 225 integer (225 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Current LB:  0.0 

-1e+100
0.0
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 255 rows, 240 columns and 690 nonzeros
Model fingerprint: 0x713bdd30
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e+00, 2e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 255 rows and 240 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective 

Extra 8 simplex iterations after uncrush
      20    4.3029000e+04   0.000000e+00   0.000000e+00      0s

Solved in 20 iterations and 0.01 seconds
Optimal objective  4.302900000e+04
[0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 2.0, 6.0, 1.0, 1.0]
Current LB:  0.0 

0.0
0.0
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 255 rows, 240 columns and 690 nonzeros
Model fingerprint: 0x0b44555a
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e+00, 2e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 238 rows and 200 columns
Presolve time: 0.00s
Presolved: 17 rows, 40 columns, 68 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.0021000e+04   4.450000e+01   0.000000e+00      0s
Extra 2 simplex iterations after uncrush
      18    4.0779000e+04   0.000000e+00   0.000000e+00      0s

Solved in 18 iterations and 0.01 seconds
Optimal objective  4.07790

Presolved: 28 rows, 81 columns, 146 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.2750000e+03   5.650000e+01   0.000000e+00      0s
      27    3.2750000e+03   0.000000e+00   0.000000e+00      0s

Solved in 27 iterations and 0.01 seconds
Optimal objective  3.275000000e+03
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Current LB:  35627.0 

0.0
35627.0
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 255 rows, 240 columns and 690 nonzeros
Model fingerprint: 0x29d1e20c
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e+00, 2e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 227 rows and 159 columns
Presolve time: 0.01s
Presolved: 28 rows, 81 columns, 146 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.2750000e+03   5.650000e+01   0.000000e+00      0s
      25    3.1670000e+03   0.0

Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e+00, 2e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 241 rows and 211 columns
Presolve time: 0.00s
Presolved: 14 rows, 29 columns, 42 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.4564000e+04   3.800000e+01   0.000000e+00      0s
      10    7.0437000e+04   0.000000e+00   0.000000e+00      0s

Solved in 10 iterations and 0.01 seconds
Optimal objective  7.043700000e+04
[0.0, 8.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 9.0, 0.0, 0.0, 0.0, 1.0, 0.0]
Current LB:  25234.99999999994 

7105.028836808588
25234.99999999994
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 255 rows, 240 columns and 690 nonzeros
Model fingerprint: 0xc74716a8
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e+00, 2e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+01]
Presolve re

Optimize a model with 255 rows, 240 columns and 690 nonzeros
Model fingerprint: 0x3a61a202
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e+00, 2e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 245 rows and 219 columns
Presolve time: 0.01s
Presolved: 10 rows, 21 columns, 28 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.6093000e+04   1.550000e+01   0.000000e+00      0s
       7    6.2199000e+04   0.000000e+00   0.000000e+00      0s

Solved in 7 iterations and 0.01 seconds
Optimal objective  6.219900000e+04
[0.0, 0.0, 0.0, 0.0, 13.0, 0.0, 0.0, 0.0, 10.0, 1.0, 0.0, 5.0, 0.0, 1.0, 0.0]
Current LB:  12540.000000000004 

7395.330203802627
12540.000000000004
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 255 rows, 240 columns and 690 nonzeros
Model fingerprint: 0xd05d5c85
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [

...

Model fingerprint: 0x9cad6a84
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e+00, 2e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 239 rows and 209 columns
Presolve time: 0.00s
Presolved: 16 rows, 31 columns, 46 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.7940000e+03   4.000000e+01   0.000000e+00      0s
      13    2.1061000e+04   0.000000e+00   0.000000e+00      0s

Solved in 13 iterations and 0.01 seconds
Optimal objective  2.106100000e+04
[0.0, 0.0, 0.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0]
 101918 68380 9258.89582   35   31 12126.0000 9064.26537  25.2%   9.2  231s
Current LB:  12080.000000000005 

9065.804820781881
12080.000000000005
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 255 rows, 240 columns and 690 nonzeros
Model fingerprint: 0x76760390
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Obje

Presolve time: 0.00s
Presolved: 13 rows, 23 columns, 34 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.8020000e+03   3.150000e+01   0.000000e+00      0s
      12    1.8857000e+04   0.000000e+00   0.000000e+00      0s

Solved in 12 iterations and 0.01 seconds
Optimal objective  1.885700000e+04
[0.0, 0.0, 0.0, 4.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0]
 113417 78471 9959.72014   61   24 12126.0000 9072.49877  25.2%   9.6  260s
Current LB:  12070.999999999985 

9073.125716605873
12070.999999999985
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 255 rows, 240 columns and 690 nonzeros
Model fingerprint: 0x574a8ad2
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e+00, 2e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 243 rows and 216 columns
Presolve time: 0.00s
Presolved: 12 rows, 24 columns, 32 nonzeros

Iteration    Objective      

[2.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
 120650 84429 9336.43127   45   25 12126.0000 9077.54665  25.1%   9.7  280s
 123240 86628 10699.7511   86   13 12126.0000 9079.27927  25.1%   9.8  285s
Current LB:  11827.999999999993 

9080.107256150466
11827.999999999993
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 255 rows, 240 columns and 690 nonzeros
Model fingerprint: 0x762c10a6
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e+00, 2e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 243 rows and 218 columns
Presolve time: 0.00s
Presolved: 12 rows, 22 columns, 32 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.3079000e+04   1.150000e+01   0.000000e+00      0s
Extra 20 simplex iterations after uncrush
      29    1.7167000e+04   0.000000e+00   0.000000e+00      0s

Solved in 29 iterations and 0.01 seconds
Optimal obj


Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.6790000e+03   2.850000e+01   0.000000e+00      0s
      10    1.4941000e+04   0.000000e+00   0.000000e+00      0s

Solved in 10 iterations and 0.01 seconds
Optimal objective  1.494100000e+04
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0]
 141771 102592 9394.65394   48   29 12126.0000 9087.06972  25.1%  10.2  335s
 144247 104803 9119.85449   39   25 12126.0000 9087.68821  25.1%  10.2  341s
 146759 107011 10497.5340   93   21 12126.0000 9088.65528  25.0%  10.3  346s
 149265 109116 12123.9554  105    5 12126.0000 9089.82112  25.0%  10.3  351s
Current LB:  12122.000000000002 

9090.416374720353
12122.000000000002
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 255 rows, 240 columns and 690 nonzeros
Model fingerprint: 0xcf1d94d4
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e+00, 2e+03]
  Bounds range     [0e+00, 0e+00]
  R

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [10]:
k = 1
mvars = m.getVars()
y0 = np.zeros((locations, customers))
for i in range(locations):
    for j in range(customers):
        y0[i,j] = mvars[k].x
        k+=1

In [11]:
model0 = gp.Model()
x0 = {}
for i in range(locations):
    for j in range(customers):
        x0[i,j] = model0.addVar(obj=varcost[i][j])

#demand constraint
demand_constr = {} 
for j in range(customers):
    demand_constr[j] = model0.addConstr(sum(x0[i,j] for i in range(locations)) == demand[j])
        
#capacity constraint
cap_constr = {}
for i in range(locations):
    cap_constr[i] = model0.addConstr(sum(x0[i,j] for j in range(customers)) <= capacity[i])

#logical constraint
log_constr = {}
for i in range(locations):
    for j in range(customers):
        log_constr[i,j] = model0.addConstr(x0[i,j] <= y0[i,j] * min(capacity[i], demand[j]))

model0.optimize()

print("Objective: " + str(model0.objVal))

print("Solution:")
import pandas as pd
index = ['Source ' + str(x) for x in range(locations)]
columns = ['Customer ' + str(x) for x in range(customers)]

solution = pd.DataFrame(index=index, columns=columns)

for i in range(locations):
    for j in range(customers):
        solution.iloc[i,j] = x0[i,j].x

print(solution)

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 255 rows, 225 columns and 675 nonzeros
Model fingerprint: 0xcaf3d292
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e+00, 4e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 255 rows and 225 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.2210000e+03   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  3.221000000e+03
Objective: 3221.0
Solution:
          Customer 0 Customer 1 Customer 2 Customer 3 Customer 4 Customer 5  \
Source 0           0          0          0          0         13          6   
Source 1           0          0          0          0          0          0   
Source 2           0         17          0          0          0          0   
Source 3          15          0          0  

In [12]:
model0.objVal+sum([fixcost[i][j] for i in range(locations) for j in range(customers) if x0[i,j].x > 0])

12245.0

In [13]:
sum(fixcost[i][j] for i in range(locations) for j in range(customers) if transport[i,j] > 0)

8838